In [22]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Activation, LSTM
from keras.optimizers import SGD, Adam, RMSprop
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras import regularizers
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import cv2
import random

font = {'weight' : 'bold',
        'size'   : 12}

matplotlib.rc('font', **font)

In [23]:
!wget -O kafka_Metamorphosis.txt https://www.gutenberg.org/files/5200/5200-0.txt

--2023-04-06 07:05:51--  https://www.gutenberg.org/files/5200/5200-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 142017 (139K) [text/plain]
Saving to: ‘kafka_Metamorphosis.txt’

kafka_Metamorphosis 100%[===================>] 138.69K  --.-KB/s    in 0.1s    

2023-04-06 07:05:51 (1.27 MB/s) - ‘kafka_Metamorphosis.txt’ saved [142017/142017]



In [24]:
text = open('kafka_Metamorphosis.txt', 'r').read().lower()
print('text length: ', len(text))

text length:  138408


In [25]:
print(text[10000:11000])

 too hard to move; it
went so slowly; and finally, almost in a frenzy, when he carelessly
shoved himself forwards with all the force he could gather, he chose
the wrong direction, hit hard against the lower bedpost, and learned
from the burning pain he felt that the lower part of his body might
well, at present, be the most sensitive.

so then he tried to get the top part of his body out of the bed first,
carefully turning his head to the side. this he managed quite easily,
and despite its breadth and its weight, the bulk of his body eventually
followed slowly in the direction of the head. but when he had at last
got his head out of the bed and into the fresh air it occurred to him
that if he let himself fall it would be a miracle if his head were not
injured, so he became afraid to carry on pushing himself forward the
same way. and he could not knock himself out now at any price; better
to stay in bed than lose consciousness.

it took just as much effort to get back to where he had be

In [26]:
chars = sorted(list(set(text)))
print('total chars: ', len(chars))

total chars:  62


In [27]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [28]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i + maxlen])
  next_chars.append(text[i + maxlen: i + maxlen + 1])

print('nb sentences: ', len(sentences))

nb sentences:  46123


In [29]:
print(sentences[:3])
print(next_chars[:3])

['\ufeffthe project gutenberg ebook of metamorp', 'e project gutenberg ebook of metamorphos', 'roject gutenberg ebook of metamorphosis,']
['h', 'i', ' ']


In [30]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

<ipython-input-30-8b86d899b220>:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
<ipython-input-30-8b86d899b220>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


In [31]:
print(x[:3])
print(y[:3])

[[[False False False ... False False  True]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False  True False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]]
[[False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False Fals

In [32]:
model = Sequential()
model.add(LSTM(254, input_shape = (maxlen, len(chars))))
model.add(Dense(10*len(chars)))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [33]:
model.compile(loss = 'categorical_crossentropy', optimizer = RMSprop(lr = 0.01))

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [34]:
model.fit(x, y, batch_size = 128, epochs = 10)

Epoch 1/10
361/361 [==============================] - 9s 13ms/step - loss: 4.1306
Epoch 2/10
361/361 [==============================] - 4s 10ms/step - loss: 1.9681
Epoch 3/10
361/361 [==============================] - 4s 11ms/step - loss: 1.7280
Epoch 4/10
361/361 [==============================] - 4s 11ms/step - loss: 1.5686
Epoch 5/10
361/361 [==============================] - 4s 10ms/step - loss: 1.4530
Epoch 6/10
361/361 [==============================] - 3s 9ms/step - loss: 1.3474
Epoch 7/10
361/361 [==============================] - 3s 9ms/step - loss: 1.2566
Epoch 8/10
361/361 [==============================] - 4s 11ms/step - loss: 1.1687
Epoch 9/10
361/361 [==============================] - 3s 9ms/step - loss: 1.0845
Epoch 10/10
361/361 [==============================] - 3s 9ms/step - loss: 1.0113


In [48]:
def sample(preds, temperature = 1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(i, preds, 1)
  return np.argmax(probas)

def generate_text(sentence, length, diversity):
  generated = ''
  generated += sentence
  for i in range(length):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
      x_pred[0, t, char_indices[char]] = 1.

    pred = model.predict(x_pred, verbose = 0)[0]
    next_index = sample(pred, diversity)
    next_char = indices_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char
  
  return generated

In [55]:
text = " the first thing he wanted to create is "
len(text)

40

In [56]:
sentence = text[0: maxlen]

print(generate_text(sentence, 30, 0.2))

 the first thing he wanted to create is if if we would be seen in this
